### **configurable**

다양한 옵션(Model, Prompt)을 동적으로 설정 할 수 있는 방법 

- `configurable_fields` 
    - 실행 가능한 객체의 특정 필드 구성

- `configurable_alternatives`
    - 런타임 중에 설정할 수 있는 특정 실행 가능한 객체애 대한 대안 나열

### **configurable_fields**

시스템의 설정 값을 정의하는 필드

- 시스템의 동작 제어, 사용자의 요구사항에 맞는 시스템 구성
- 설정 파일, 환경 변수, 사용자 인터페이스를 통해 제공
- DB연결 정보, 로깅 설정, 보안 옵션, 파라미터 튜닝

In [3]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

In [ ]:
# 기본 사용 했을때의 정보 
model = ChatOpenAI(temperature=0, model_name="gpt-4o")

model.invoke("대한민국의 수도는 어디야?").__dict__

{'content': '대한민국의 수도는 서울이야.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 17,
   'prompt_tokens': 22,
   'total_tokens': 39,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-3.5-turbo-0125',
  'system_fingerprint': None,
  'id': 'chatcmpl-BGQR68TLLw7wfI4G16oD5jfM8BfNJ',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-1ad8fd22-7482-4028-99da-c4b9d2297a00-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 22,
  'output_tokens': 17,
  'total_tokens': 39,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 0}}}

In [ ]:
# Model Name 동적 정의
model = ChatOpenAI().configurable_fields(
    model_name = ConfigurableField(
    id = 'gpt_version',
    name = 'Version of GPT',
    description = 'Get the version of GPT ex) gpt-4o, gpt-3.5-turbo, etc'
    )
)

In [44]:
model.invoke(
    "대한민국의 수도는 어디야?",
    config={"configurable": {"gpt_version": "gpt-4o"}},
).response_metadata['model_name']

'gpt-4o-2024-08-06'

In [45]:
model.invoke(
    "대한민국의 수도는 어디야?",
    config={"configurable": {"gpt_version": "gpt-4o-mini"}},
).response_metadata['model_name']

'gpt-4o-mini-2024-07-18'

In [46]:
# 모델에서 정의 
model.with_config(configurable={"gpt_version": "gpt-4o-mini"}).invoke(
    "대한민국의 수도는 어디야?"
).response_metadata['model_name']

'gpt-4o-mini-2024-07-18'

### **Chain 설정**

In [ ]:
prompt = PromptTemplate.from_template('{topic}에 대해 설명해줘')
chain = prompt | model

In [ ]:
# invoke를 호출 하면서 configurable을 설정하여 모델 변경
chain.invoke('대한민국수도', config={"configurable": {"gpt_version": "gpt-4o-mini"}}).response_metadata['model_name']

'gpt-4o-mini-2024-07-18'

In [ ]:
# 체인에서 모델을 정의하고 invoke로 llm 호출
chain.with_config(configurable={"gpt_version": "gpt-4o-mini"}).invoke("대한민국의 수도").response_metadata['model_name']

'gpt-4o-mini-2024-07-18'

### **Configurable Alternatives: Runnable 객체 자체의 대안 설정**

- 서로다른 모델 API 과 Prompt를 사용 가능하게 함

In [61]:
from langchain.prompts import PromptTemplate
from langchain_groq import  ChatGroq
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=0, model="gpt-4o"
).configurable_alternatives(
    
    # id 설정    
    ConfigurableField(id="llm"),

    # default 설정
    default_key="anthropic",
    
    # 대체 모델 설정(gpt-4o-mini, gemma2-9b-it)
    mini = ChatOpenAI(model="gpt-4o-mini"),
    
    gemma = ChatGroq(model ="gemma2-9b-it")
    
)
prompt = PromptTemplate.from_template("{topic} 에 대해 간단히 설명해주세요.")
chain = prompt | llm

In [58]:
# default 설정으로 실행
chain.invoke('뉴진스').response_metadata['model_name']

'gpt-4o-2024-08-06'

In [59]:
# 체인에서 모델을 정의하고 invoke로 llm 호출
chain.with_config(configurable={'llm' : 'gemma'}).invoke('뉴진스').response_metadata['model_name']

'gemma2-9b-it'

In [63]:
# chain.invoke('뉴진스', configurable={'llm' : 'mini'}).response_metadata['model_name'] => 이거 안됨 => with_config로 설정해야함
chain.with_config(configurable={'llm' : 'mini'}).invoke('뉴진스').response_metadata['model_name']

'gpt-4o-mini-2024-07-18'

### **Prompt 대안 설정**

In [78]:
llm = ChatOpenAI(model = 'gpt-4o')

prompt = PromptTemplate.from_template("{country} 의 수도는 어디야?").configurable_alternatives(
    
    # id 설정
    ConfigurableField(id="prompt"),

    # default 설정
    default_key="capital",

    # 여러가지 프롬프트 설정
    area=PromptTemplate.from_template("{country} 의 면적은 얼마야?"),
    population=PromptTemplate.from_template("{country} 의 인구는 얼마야?"),
    eng=PromptTemplate.from_template("{input} 을 영어로 번역해주세요. 번역된 결과만 제공해주세요"),
)

chain = prompt | llm

In [74]:
# default 설정
chain.invoke('대한민국').content

'대한민국의 수도는 서울특별시입니다.'

In [75]:
chain.with_config(configurable={'prompt' : 'area'}).invoke('대한민국').content

'대한민국의 면적은 약 100,210 평방킬로미터입니다.'

In [76]:
chain.with_config(configurable={'prompt' : 'population'}).invoke('대한민국').content

'2023년 기준으로 대한민국의 인구는 약 5,100만 명 정도입니다. 정확한 인구 수치는 시간에 따라 변동될 수 있으므로 최신 통계를 참고하는 것이 좋습니다.'

In [79]:
chain.with_config(configurable={'prompt' : 'eng'}).invoke('안녕하세요 반가워요 나는 지금 LangChain 공부중입니다').content

'Hello, nice to meet you. I am currently studying LangChain.'

### **Model & Prompt**

In [80]:
llm = ChatOpenAI(
    temperature=0, model="gpt-4o"
).configurable_alternatives(
    
    # id 설정    
    ConfigurableField(id="llm"),

    # default 설정
    default_key="anthropic",
    
    # 대체 모델 설정(gpt-4o-mini, gemma2-9b-it)
    mini = ChatOpenAI(model="gpt-4o-mini"),
    
    gemma = ChatGroq(model ="gemma2-9b-it")
    
)

prompt = PromptTemplate.from_template("{country} 의 수도는 어디야?").configurable_alternatives(
    
    # id 설정
    ConfigurableField(id="prompt"),

    # default 설정
    default_key="capital",

    # 여러가지 프롬프트 설정
    area=PromptTemplate.from_template("{country} 의 면적은 얼마야?"),
    population=PromptTemplate.from_template("{country} 의 인구는 얼마야?"),
    eng=PromptTemplate.from_template("{input} 을 영어로 번역해주세요. 번역된 결과만 제공해주세요"),
)

chain = prompt | llm

In [86]:
# default 설정
response = chain.invoke('대한민국')
response.content, response.response_metadata['model_name']

('대한민국의 수도는 서울입니다.', 'gpt-4o-2024-08-06')

In [ ]:
# LLM, Prompt 동시 설정
response = chain.with_config(configurable = {'prompt' : 'area', 'llm' : 'mini'}).invoke('대한민국')
response.content, response.response_metadata['model_name']

('대한민국의 면적은 약 100,210 평방킬로미터입니다. 이는 한반도의 남쪽 부분에 해당하며, 국토의 대부분이 산지와 평지로 구성되어 있습니다.',
 'gpt-4o-mini-2024-07-18')

In [88]:
# LLM만 설정
response = chain.with_config(configurable = {'llm' : 'gemma'}).invoke('대한민국')
response.content, response.response_metadata['model_name']

('대한민국의 수도는 **서울**입니다. 😊  \n', 'gemma2-9b-it')

In [89]:
# Prompt만 설정
response = chain.with_config(configurable = {'prompt' : 'population'}).invoke('대한민국')
response.content, response.response_metadata['model_name']

('2023년 기준으로 대한민국의 인구는 약 5,100만 명 정도로 추산됩니다. 정확한 수치는 통계청이나 관련 기관의 최신 자료를 참조하는 것이 좋습니다.',
 'gpt-4o-2024-08-06')

### **설정 저장**

In [90]:
gpt4o_mini = chain.with_config(configurable = {'llm' : 'mini'})
response = gpt4o_mini.invoke('대한민국')
response.content, response.response_metadata['model_name']

('대한민국의 수도는 서울입니다.', 'gpt-4o-mini-2024-07-18')